# Acquire Data, Preprocess and plot - without thunder

## 1. Get user input

In [9]:
# Provide Folder from which to get data
foldername = '/Users/seetha/Desktop/Habenula/Data/Fish104/Block1/'

# Image sizes
img_size_x = 512
img_size_y = 512
img_size_crop_x1 = 290
img_size_crop_x2 = 20
img_size_crop_y1 = 20
img_size_crop_y2 = 20

# Background subtraction ROI
bg_roi = 0

# Stimulus on and off time
stimulus_on_time = [46, 98, 142, 194]
stimulus_off_time = [65, 117, 161, 213]

## 2. Libraries

### Import modules required by this function

In [2]:
import os, sys
MODULES_DIR = '/Users/seetha/Desktop/my_cool_python_functions/for_plotting/'
sys.path.append(os.path.dirname(MODULES_DIR))
MODULES_DIR = '/Users/seetha/Desktop/my_cool_python_functions/for_opening_data/'
sys.path.append(os.path.dirname(MODULES_DIR))

### Import libraries

In [3]:
from libtiff import TIFF
from tifffile import TiffFile
import numpy as np
import os
import PIL.Image as pil
import math
import matplotlib.pyplot as plt
import seaborn as sns

### Import user defined libraries

In [ ]:
from open_process_data import PreprocessData
OpenPreprocessData = PreprocessData(stimulus_on_time, stimulus_off_time)

In [4]:
from plotting_functions import FormatForPlotting
Format_to_plot = FormatForPlotting(stimulus_on_time, stimulus_off_time)
heatmapplotting = Format_to_plot.heatmapplotting
linegraphplotting = Format_to_plot.linegraphplotting

In [5]:
#Import Thunder libraries
from thunder import Colorize
image = Colorize.image

In [6]:
#Matplotlib plots to be inline in the notebook
%matplotlib inline

### Define functions

In [10]:
# Plot planes in the given image as subplot
def PlotImagePlanes(img):
    print 'Image Size = ' + str(np.shape(data_raw))
    numz = np.size(img,2)
    num_subplots = int((math.ceil(numz / 2.) * 2)/2)
    fig1 = plt.figure(figsize=(15,10))
    for ii in xrange(0, numz):    
        ax1 = fig1.add_subplot(num_subplots,2,ii+1)
        ax1 = image(np.mean(img[:,:,ii,:],2))    
        plt.title('Plane: ' + str(ii+1), fontsize=12, color='white')
    plt.show()

In [9]:
def get_tif_images_raw(data, lst, onlyfiles, text_file, tif, 
                       img_size_x, img_size_y, img_size_crop_x1, img_size_crop_x2, img_size_crop_y1, img_size_crop_y2):
    count_z = 0

    # Store tiff in numpy array data
    for image in tif.iter_images():

        ##Resizing if required
        if np.size(image, 1) != img_size_y or np.size(image, 0) != img_size_x:
            if lst == 1 and count_z == 0:
                print "Resizing image..."
            temp_image = pil.Image.fromarray(image)
            temp_image1 = np.array(temp_image.resize((img_size_y, img_size_x), pil.Image.NEAREST))
            temp_image1.transpose()
        else:
            temp_image1 = image

        # Cropping unwanted pixels if required
        if img_size_crop_x1 != 0 and img_size_crop_y1 != 0:
            if lst == 1 and count_z == 0:
                print "Cropping x and y pixels.."
            temp_image2 = temp_image1[img_size_crop_x1:-img_size_crop_x2, img_size_crop_y1:-img_size_crop_y2]
        elif img_size_crop_x1 != 0 and img_size_crop_y1 == 0:
            if lst == 1 and count_z == 0:
                print "Cropping only x pixels.."
            temp_image2 = temp_image1[img_size_crop_x1:-img_size_crop_x2, :]
        elif img_size_crop_x1 == 0 and img_size_crop_y1 != 0:
            if lst == 1 and count_z == 0:
                print "Cropping only x pixels.."
            temp_image2 = temp_image1[:, img_size_crop_y1:-img_size_crop_y2]
        else:
            temp_image2 = temp_image1

        data[:, :, count_z, lst - 1] = temp_image2
        count_z = count_z + 1

    return data

## 2. Get tiff files from folder

In [12]:
# Get names of all tiff files in the directory
onlyfiles = [f for f in os.listdir(foldername) 
             if (os.path.isfile(os.path.join(foldername, f)) 
                 and f.find('.tif') > 0 and f.find('T=') >= 0)]


for lst in xrange(1, np.size(onlyfiles, axis=0) + 1):
    tifffile_name = foldername + 'T=' + str(lst) + '.tif'
    with TiffFile(tifffile_name) as tif:
        tif1 = tif.asarray()
        
        if len(np.shape(tif1)) == 3:
            tif1 = tif1.transpose((1,2,0))
            count_z = np.size(tif1,2)
            
            
        if lst == 1:
            data_filtered = np.zeros((img_size_x - (img_size_crop_x1 + img_size_crop_x2),
                                      img_size_y - (img_size_crop_y1 + img_size_crop_y2), count_z,
                                      np.size(onlyfiles, 0)), dtype=np.uint8)
            

            print np.shape(data_filtered)
            print count_z
            print np.shape(tif1)

(202, 472, 5, 311)
5
(512, 512, 5)


In [ ]:
if lst == 1:
        count_z = np.size(tif1,0)

        data_raw = np.zeros((img_size_x, img_size_y, count_z, np.size(onlyfiles, 0)), dtype=np.uint8)
        
    data_raw = get_tif_images_raw(data_raw, lst, onlyfiles, tif1, 
                                  img_size_x, img_size_y, img_size_crop_x1, img_size_crop_x2, img_size_crop_y1,
                                  img_size_crop_y2, bg_roi)

### Plot planes

In [ ]:
PlotImagePlanes(data_raw)

## 3. Plot heatmaps for each plane

In [ ]:
img = np.asarray(data_raw[:,:,1,:])
img_reshape = np.reshape(img, (np.size(img,0)*np.size(img,1), np.size(img, 2)))
img_sort = np.argsort(np.mean(img_reshape, axis=1))
img_sort = img_reshape[img_sort, :][-5000:, :]

In [ ]:
fs = plt.figure(figsize=(10, 5))
gs = plt.GridSpec(4, 2, width_ratios=[9, 1])
ax1 = fs.add_subplot(gs[:2, :1])
im = plt.imshow(img_sort, aspect='auto', cmap='jet')  
heatmapplotting(ax1, num_bins=4)

In [ ]:
ax2 = fs.add_subplot(gs[:2, 1:])
color_bar = plt.colorbar(im, cax=ax2)
cbytick_obj = plt.getp(color_bar.ax.axes, 'yticklabels')  
plt.setp(cbytick_obj, color='w')
    
fs = plt.figure(figsize=(10, 5))
gs = plt.GridSpec(4, 2, width_ratios=[9, 1])
ax2 = fs.add_subplot(gs[:2, :1])
ax2.plot(np.mean(img_sort,axis=0))
linegraphplotting(ax2, xlim_input=(0, np.size(img_sort, 1)), num_bins=4)
plt.show()

In [ ]:
img_reshape = np.reshape